In [1]:
import googlemaps
import polyline
import time
import matplotlib.pyplot as plt
import numpy as np
import config


# Initialize with your Google Maps API key
gmaps = googlemaps.Client(key=config.api_key)


start_address = "301 Pike St, Seattle, WA 98101"
end_address = "423 Terry Ave, Seattle, WA 98104"


# Step 1: Get exact coordinates of start and end
start_location = gmaps.geocode(start_address)[0]["geometry"]["location"]
end_location = gmaps.geocode(end_address)[0]["geometry"]["location"]


# Step 2: Get the main route
directions = gmaps.directions(
    origin=start_location,
    destination=end_location,
    mode="walking",
    alternatives=False
)


route = directions[0]
points = polyline.decode(route["overview_polyline"]["points"])

In [3]:
points

[(47.60968, -122.33701),
 (47.60905, -122.33644),
 (47.60908, -122.33639),
 (47.60925, -122.33597),
 (47.60927, -122.33599),
 (47.60936, -122.33577),
 (47.60934, -122.33575),
 (47.60942, -122.33555),
 (47.60952, -122.33532),
 (47.60971, -122.33483),
 (47.60974, -122.33485),
 (47.60982, -122.33469),
 (47.60979, -122.33467),
 (47.60997, -122.33426),
 (47.61002, -122.33408),
 (47.61018, -122.33371),
 (47.61023, -122.33375),
 (47.61027, -122.33361),
 (47.61027, -122.33353),
 (47.61017, -122.33343),
 (47.61014, -122.3334),
 (47.61013, -122.33342),
 (47.61004, -122.33333),
 (47.60994, -122.33325),
 (47.60939, -122.33274),
 (47.60927, -122.33262),
 (47.60931, -122.33252),
 (47.60926, -122.33247),
 (47.6092, -122.33258),
 (47.60915, -122.3325),
 (47.60883, -122.33223),
 (47.60856, -122.332),
 (47.60858, -122.33196),
 (47.60855, -122.33193),
 (47.60848, -122.33181),
 (47.60845, -122.33187),
 (47.60845, -122.3318),
 (47.60839, -122.3318),
 (47.60839, -122.33186),
 (47.60828, -122.33173),
 (47.60

In [4]:
# Step 3: Get elevation data in chunks (Google Elevation API limit = 512 samples)
# path = "|".join(f"{lat},{lng}" for lat, lng in points)
# Ensure points are lat/lon and >= 2
path = [(pt[0], pt[1]) for pt in points]

In [5]:
path

[(47.60968, -122.33701),
 (47.60905, -122.33644),
 (47.60908, -122.33639),
 (47.60925, -122.33597),
 (47.60927, -122.33599),
 (47.60936, -122.33577),
 (47.60934, -122.33575),
 (47.60942, -122.33555),
 (47.60952, -122.33532),
 (47.60971, -122.33483),
 (47.60974, -122.33485),
 (47.60982, -122.33469),
 (47.60979, -122.33467),
 (47.60997, -122.33426),
 (47.61002, -122.33408),
 (47.61018, -122.33371),
 (47.61023, -122.33375),
 (47.61027, -122.33361),
 (47.61027, -122.33353),
 (47.61017, -122.33343),
 (47.61014, -122.3334),
 (47.61013, -122.33342),
 (47.61004, -122.33333),
 (47.60994, -122.33325),
 (47.60939, -122.33274),
 (47.60927, -122.33262),
 (47.60931, -122.33252),
 (47.60926, -122.33247),
 (47.6092, -122.33258),
 (47.60915, -122.3325),
 (47.60883, -122.33223),
 (47.60856, -122.332),
 (47.60858, -122.33196),
 (47.60855, -122.33193),
 (47.60848, -122.33181),
 (47.60845, -122.33187),
 (47.60845, -122.3318),
 (47.60839, -122.3318),
 (47.60839, -122.33186),
 (47.60828, -122.33173),
 (47.60